<a href="https://colab.research.google.com/github/hygo2025/rna/blob/main/especificacao-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Trabalho 1: Diferenciação Automática com Grafos Computacionais

## Informações Gerais

- Data de Entrega: 29/06/2025
- Pontuação: 10 pontos (+4 pontos extras)
- O trabalho deve ser feito individualmente.
- A entrega do trabalho deve ser realizada via sistema testr.



## Especificação

⚠️ *Esta explicação assume que você leu e entendeu os slides sobre grafos computacionais.*

O trabalho consiste em implementar um sistema de diferenciação automática usando grafos computacionais e utilizar este sistema para resolver um conjunto de problemas.

Para isto, devem ser definidos um tipo Tensor para representar dados (similares aos arrays do numpy) e operações (e.g., soma, subtração, etc.) que geram tensores como saída.

Sempre que uma operação é realizada, é armazenado no tensor de saída referências para os seus pais, isto é, os valores usados como entrada para a operação.


### Imports

In [1]:
!pip install toolz

In [2]:
from typing import Optional, Union, Any
from collections.abc import Iterable
from abc import ABC, abstractmethod
import numbers

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from toolz import pipe

sns.set_style('whitegrid')

### Classe NameManager

A classe NameManager provê uma forma conveniente de dar nomes intuitivos para tensores que resultam de operações. A idéia é tornar mais fácil para o usuário das demais classes qual operação gerou qual tensor. Ela provê os seguintes métodos públicos:

- reset(): reinicia o sistema de gestão de nomes.
- new(<basename>: str): retorna um nome único a partir do nome de base passado como argumento.
  
Como indicado no exemplo abaixo da classe, a idéia geral é que uma sequência de operações é feita, os nomes dos tensores sejam os nomes das operações seguidos de um número. Se forem feitas 3 operações de soma e uma de multiplicação, seus tensores de saída terão os nomes "add:0", "add:1", "add:2" e "prod:0".

In [3]:

class NameManager:
    _counts = {}

    @staticmethod
    def reset():
        NameManager._counts = {}

    @staticmethod
    def _count(name):
        if name not in NameManager._counts:
            NameManager._counts[name] = 0
        count = NameManager._counts[name]
        return count

    @staticmethod
    def _inc_count(name):
        assert name in NameManager._counts, f'Name {name} is not registered.'
        NameManager._counts[name] += 1

    @staticmethod
    def new(name: str):
        count = NameManager._count(name)
        tensor_name = f"{name}:{count}"
        NameManager._inc_count(name)
        return tensor_name

# exemplo de uso
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('add'))
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('prod'))

NameManager.reset()

add:0
in:0
add:1
add:2
in:1
prod:0


### Classe Tensor

Deve ser criada uma classe `Tensor` representando um array multidimensional.

In [4]:

from toolz import pipe

InputArr = Union[np.ndarray, list, numbers.Number, Any]

def standardize_tensor(arr: InputArr) -> np.ndarray:
  match arr:
    case Tensor():
      return arr.numpy().copy()

    case list() | numbers.Number():
      return np.array(arr, dtype=float)

    case np.ndarray():
      return arr.astype(float).copy()

    case _:
      raise TypeError(f"Tipo de dado não suportado: {type(arr)}")

def standardize_dimensions(arr: np.ndarray) -> np.ndarray:
  match arr.ndim:
    case 0:
      return arr.reshape((1, 1))  # escalar
    case 1:
      return arr.reshape(-1, 1) # vetor coluna
    case 2:
      return arr
    case _:
        raise ValueError(f"Dimensões não suportadas: {arr.ndim}. Máximo 2 dimensões.")

class Tensor:
  def __init__(self,
         arr: InputArr,
         parents: list[Any] = None,
         requires_grad: bool = True,
         name: str = '',
         operation=None):

    self.grad = None
    self._arr = pipe(arr, standardize_tensor, standardize_dimensions)
    self.requires_grad = requires_grad
    self._operation = operation
    self._name = name
    self._parents = parents or []



  def __neg__(self):
    negated_arr = -self.numpy()
    return Tensor(negated_arr, requires_grad=False)

  def zero_grad(self):
    """Reinicia o gradiente com zero"""
    if self.requires_grad: #Aqui fala pa reiniciar o gradiente com zero, entretante esse grad deve ser um um tensor como no enunciado ou um np.array?
      # self._grad = np.zeros_like(self._arr)
      self.grad = Tensor(np.zeros_like(self._arr), requires_grad=False, name=f"{self._name}_grad")

  def numpy(self) -> np.ndarray:
    """Retorna o array interno"""
    return self._arr

  def __repr__(self):
    """Permite visualizar os dados do tensor como string"""
    return f"Tensor({self._arr}, name={self._name}, shape={self._arr.shape})"

  def backward(self, my_grad: 'Tensor' =None):
    """Método usado tanto iniciar o processo de
    diferenciação automática, quanto por um filho
    para enviar o gradiente do pai. No primeiro
    caso, o argumento my_grad não será passado.
    """

    # Ainda nao entendi pq eu nao iria querer o grad, acho que só é valido para o ultimo elemnto do grafo
    if not self.requires_grad:
        return

    if my_grad is None: # Se for o final do grafo retona tudo 1
        my_grad = Tensor(np.ones_like(self._arr), requires_grad=False)

    if self.grad is None: # Se for o primeiro gradiente, inicializa, nao da da somar None com um Tensor
        self.grad = my_grad
    else:
        self.grad._arr += my_grad.numpy() # Aqui acredito que ocorre o acumumulo do gradiente ponto a ponto

    if self._operation:
      # O professor explicou na aula em relacao aos parametros nao nomeados, acho que devo passar, pois na classe Op o metodo grad recebe args que sao os pais
      parent_grads = self._operation.grad(self.grad, *self._parents) # faz o calculo da operacao, por exemplo da soma
      for parent, parent_grad_arr in zip(self._parents, parent_grads):
        parent.backward(Tensor(parent_grad_arr))


### Interface de  Operações

A classe abaixo define a interface que as operações devem implementar. Ela não precisa ser modificada, mas pode, caso queira.

In [5]:

class Op(ABC):
    @abstractmethod
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando as entradas e
            retorna o tensor resultado. O método deve
            garantir que o atributo parents do tensor
            de saída seja uma lista de tensores."""

    @abstractmethod
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna os gradientes dos pais em como tensores.

        Arguments:

        - back_grad: Derivada parcial em relação à saída
            da operação backpropagada pelo filho.

        - args: variaveis de entrada da operacao (pais)
            como tensores.

        - O nome dos tensores de gradiente devem ter o
            nome da operacao seguido de '_grad'.
        """



In [6]:
from functools import wraps
from toolz import pipe

def preprocess_op(arity: int):
  """
  Decorador que pré-processa os argumentos para uma operação.
  Checa aridade e garante que todos os args são Tensors.
  https://book.pythontips.com/en/latest/decorators.html#nesting-a-decorator-within-a-function
  """
  def decorator(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
      # checa aridade
      if len(args) != arity:
        raise ValueError(
          f"A operação {func.__name__} espera {arity} operandos, mas recebeu {len(args)}."
        )

      # Força tudo ser Tensor
      processed_args = pipe(
        args,
        lambda op_args: map(
          lambda arg: arg if isinstance(arg, Tensor) else Tensor(arg, requires_grad=False),
          op_args
        ),
        list
      )

      # chama a função original
      return func(self, *processed_args, **kwargs)

    return wrapper
  return decorator

### Implementação das Operações

Operações devem herdar de `Op` e implementar os métodos `__call__` e `grad`.

Pelo menos as seguintes operações devem ser implementadas:



In [7]:

class Add(Op):
    """Add(a, b): a + b"""
    @preprocess_op(arity=2)
    def __call__(self, t_a: Tensor, t_b: Tensor) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

        return Tensor(
          arr=t_a.numpy() + t_b.numpy(),
          parents=[t_a, t_b],
          operation=self,
          requires_grad=t_a.requires_grad or t_b.requires_grad,
          name='add'
        )

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
        return [back_grad, back_grad]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
add = Add()

In [8]:

class Sub(Op):
  """Sub(a, b): a - b"""
  @preprocess_op(arity=2)
  def __call__(self, t_a: Tensor, t_b: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""

    return Tensor(
      arr=t_a.numpy() - t_b.numpy(),
      parents=[t_a, t_b],
      operation=self,
      requires_grad=t_a.requires_grad or t_b.requires_grad,
      name='sub'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

    return [back_grad, -back_grad]  # A derivada de a - b em relação a a é 1 e em relação a b é -1

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sub = Sub()

In [9]:

class Prod(Op):
  """Prod(a, b): produto ponto a ponto de a e b ou produto escalar-tensor"""
  @preprocess_op(arity=2)
  def __call__(self, t_a: Tensor, t_b: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.multiply(t_a.numpy(), t_b.numpy()),
      parents=[t_a, t_b],
      operation=self,
      requires_grad=t_a.requires_grad or t_b.requires_grad,
      name='prod'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    # f(x) = u(x) * v(x)
    # f(x) = u'(x) * v(x) + u(x) * v'(x)
    a, b = args

    # a é u(x)
    # b é v(x)
    # f(x) = a * b até aqui normal

    # primeiro vou fazer a derivada em relação a `a`
    # f(x) = a * b e `b` é constante
    # f'(x) = b em relacao a a

    # agora em relação a `b`
    # f(x) = a * b e `a` é constante
    # f'(x) = a em relacao a b

    # agora é multiplicar pelo gradiante
    grad_a = np.multiply(back_grad.numpy(), b.numpy()) # gradiente que vem do pai * a derivada f'(x) = b em relacao a a
    grad_b = np.multiply(back_grad.numpy(), a.numpy()) # gradiente que vem do pai * a derivada f'(x) = a em relacao a b

    return [Tensor(grad_a, requires_grad=back_grad.requires_grad),
            Tensor(grad_b, requires_grad=back_grad.requires_grad)]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
prod = Prod()

In [10]:

class Sin(Op):
  """seno element-wise"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.sin(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='sin'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    # f(x) = sin(x)
    # f'(x) = cos(x)
    # gradiente que vem do pai * a derivada f'(x) = cos(x)
    a, = args # Args aqui é uma tupla de 1 elemento por isso tive de adicionar a `,`
    grad_a = np.multiply(back_grad.numpy(), np.cos(a.numpy()))

    return [Tensor(grad_a, requires_grad=back_grad.requires_grad)]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sin = Sin()

In [11]:

class Cos(Op):
  """cosseno element-wise"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.cos(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='cos'
    )


  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    # f(x) = cos(x)
    # f'(x) = -sin(x)
    # gradiente que vem do pai * a derivada f'(x) = -sin(x)
    a, = args # Args aqui é uma tupla de 1 elemento por isso tive de adicionar a `,`
    grad_a = np.multiply(back_grad.numpy(), -np.sin(a.numpy()))

    return [Tensor(grad_a, requires_grad=back_grad.requires_grad)]


# Instancia a classe. O objeto passa a poder ser usado como uma funcao
cos = Cos()

In [12]:

class Sum(Op):
  """Retorna a soma dos elementos do tensor"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.sum(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='sum'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

    # f(x) = x1 + x2 + x3 + ... + xn
    # f'(x) = 1 + 0 + 0 + ... + 0 em relação a x1
    # para cada elemento do tensor, a derivada é 1

    a, = args # Args aqui é uma tupla de 1 elemento por isso tive de adicionar a `,`

    grad_scalar = back_grad.numpy().item()

    # dai eu crio um novo vetor com o mesmo valor para cada elemento do tensor e a mesma forma
    grad_arr = np.full_like(a.numpy(), fill_value=grad_scalar)

    return [Tensor(grad_arr, requires_grad=back_grad.requires_grad)]


# Instancia a classe. O objeto passa a poder ser usado como uma funcao
# ⚠️ vamos chamar de my_sum porque python ja possui uma funcao sum
my_sum = Sum()

In [13]:

class Mean(Op):
  """Retorna a média dos elementos do tensor"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.mean(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='mean'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    a, = args # Args aqui é uma tupla de 1 elemento por isso tive de adicionar a `,`
    # f(x) = (x1 + x2 + x3 + ... + xn) * 1/n
    # f'(x) = 1/n + 0 + 0 + ... + 0 em relação a x1
    # para cada elemento do tensor, a derivada é 1/n
    n = a.numpy().size  # número de elementos no tensor
    grad_scalar = back_grad.numpy().item() / n  # gradiente que vem do pai dividido pelo número de elementos
    # dai eu crio um novo vetor com o mesmo valor para cada elemento do tensor e a mesma forma
    grad_arr = np.full_like(a.numpy(), fill_value=grad_scalar)
    return [Tensor(grad_arr, requires_grad=back_grad.requires_grad)]


# Instancia a classe. O objeto passa a poder ser usado como uma funcao
mean = Mean()

In [14]:

class Square(Op):
  """Eleva cada elemento ao quadrado"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.square(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='square'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    a, = args # Args aqui é uma tupla de 1 elemento por isso tive de adicionar a `,`
    # f(x) = x^2
    # f'(x) = 2x
    # para cada elemento do tensor, a derivada é 2x

    local_der = 2 * a.numpy()  # calcula 2x para cada elemento do tensor
    grad_arr = np.multiply(back_grad.numpy(), local_der)  # multiplica pelo gradiente que vem do pai

    return [Tensor(grad_arr, requires_grad=back_grad.requires_grad)]



# Instancia a classe. O objeto passa a poder ser usado como uma funcao
square = Square()

In [15]:

class MatMul(Op):
  """MatMul(A, B): multiplicação de matrizes

  C = A @ B
  de/dA = de/dc @ B^T
  de/dB = A^T @ de/dc

  """

  @preprocess_op(arity=2)
  def __call__(self, t_a: Tensor, t_b: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.matmul(t_a.numpy(), t_b.numpy()),
      parents=[t_a, t_b],
      operation=self,
      requires_grad=t_a.requires_grad or t_b.requires_grad,
      name='matmul'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

    a, b = args

    grad_a = np.matmul(back_grad.numpy(), b.numpy().T)  # de/dA = de/dc @ B^T
    grad_b = np.matmul(a.numpy().T, back_grad.numpy())  # de/dB = A^T @ de/dc

    return [Tensor(grad_a, requires_grad=back_grad.requires_grad),
        Tensor(grad_b, requires_grad=back_grad.requires_grad)]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
matmul = MatMul()

In [16]:

class Exp(Op):
  """Exponenciação element-wise"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    # https://numpy.org/doc/stable/reference/generated/numpy.exp.html
    return Tensor(
      arr=np.exp(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='exp'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    a, = args

    # f(x) = e^x onde x é cada elemento do meu tensor
    # f'(x) = e^x a derivada é ela mesma

    #entao a derivada local é aplicar o exp para cada elemento do meu tensor
    #é quase igual o square
    local_der = np.exp(a.numpy())
    grad_arr = np.multiply(back_grad.numpy(), local_der)  # multiplica pelo gradiente que vem do pai

    return [Tensor(grad_arr, requires_grad=back_grad.requires_grad)]


# Instancia a classe. O objeto passa a poder ser usado como uma funcao
exp = Exp()

In [17]:
def relu_fn(input_arr: np.ndarray) -> np.ndarray:
  return np.maximum(0, input_arr)

def relu_grad(input_arr: np.ndarray) -> np.ndarray:
    return (input_arr > 0).astype(input_arr.dtype)

class ReLU(Op):
  """ReLU element-wise"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=relu_fn(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='relu'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    a, = args

    # A relu é assim __/ entao qualquer coisa abaixo de zero é zero, acima é a identidade
    # f(x) = max(0, x)
    # f'(x) = 1 se x > 0, 0 caso contrario 0

    # Entao a derivada local é aplicar o relu_grad para cada elemento do meu tensor
    local_der = relu_grad(a.numpy())
    grad_arr = np.multiply(back_grad.numpy(), local_der)  # multiplica pelo gradiente que vem do pai

    return [Tensor(grad_arr, requires_grad=back_grad.requires_grad)]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
relu = ReLU()

In [18]:
def sigmoid_fn(input_arr: np.ndarray) -> np.ndarray:
    """
    Função sigmoid: sigma(x) = 1 / (1 + exp(-x))
    Pode ser usada com escalares, vetores ou matrizes NumPy.
    """
    return 1 / (1 + np.exp(-input_arr))

def sigmoid_grad(input_arr: np.ndarray) -> np.ndarray:
    return sigmoid_fn(input_arr) * (1 - sigmoid_fn(input_arr))

class Sigmoid(Op):
  """Sigmoid element-wise"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=sigmoid_fn(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='sigmoid'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

    a, = args

    # f(x) = 1 / (1 + exp(-x))
    # f'(x) = sigma(x) * (1 - sigma(x))

    local_der = sigmoid_grad(a.numpy())
    grad_arr = np.multiply(back_grad.numpy(), local_der)  # multiplica pelo gradiente que vem do pai

    return [Tensor(grad_arr, requires_grad=back_grad.requires_grad)]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sigmoid = Sigmoid()

In [19]:
def tanh_grad(input_arr: np.ndarray) -> np.ndarray:
  return 1 - np.square(np.tanh(input_arr))

class Tanh(Op):
  """Tanh element-wise"""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.tanh(t_a.numpy()),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='tanh'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
    a, = args

    # f(x) = tanh(x)
    # f'(x) = 1 - tanh(x)^2

    local_der = tanh_grad(a.numpy())
    grad_arr = np.multiply(back_grad.numpy(), local_der)  # multiplica pelo gradiente que vem do pai

    return [Tensor(grad_arr, requires_grad=back_grad.requires_grad)]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
tanh = Tanh()

In [20]:
def softmax_fn(input_arr: np.ndarray) -> np.ndarray:
  """Calcula a softmax de um array de valores."""

  # A função softmax é definida como:
  # softmax(x_i) = exp(x_i) / sum(exp(x_j)) para j em {1, ..., n}

  # https://victorzhou.com/blog/softmax/
  numerator = np.exp(input_arr)
  denominator = np.sum(np.exp(input_arr))

  return numerator / denominator

class Softmax(Op):
  """Softmax de um array de valores. Lembre-se que cada elemento do array influencia o resultado da função para todos os demais elementos."""
  @preprocess_op(arity=1)
  def __call__(self, t_a: Tensor) -> Tensor:
    """Realiza a operação usando os argumentos dados em args"""
    return Tensor(
      arr=np.exp(t_a.numpy())/sum(np.exp(t_a.numpy())),
      parents=[t_a],
      operation=self,
      requires_grad=t_a.requires_grad,
      name='softmax'
    )

  def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
    """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
softmax = Softmax()


### ‼️ Regras e Pontos de Atenção‼️

- Vamos fazer a hipótese simplificadora que Tensores devem ser sempre matrizes. Por exemplo, o escalar 2 deve ser armazado em `_arr` como a matriz `[[2]]`. De forma similar, a lista `[1, 2, 3]` deve ser armazenada em `_arr` como em uma matriz coluna.

- Devem ser realizados `asserts` nas operações para garantir que os shapes dos operandos fazem sentido. Esta verificação também deve ser feita depois das operações que manipulam gradientes de tensores.

- Devem ser respeitados os nomes dos atributos, métodos e classes para viabilizar os testes automáticos.

- Gradientes devem ser calculados usando uma passada pelo grafo computacional.

- Os gradientes devem ser somados e não substituídos nas chamadas de  backward. Isto vai permitir que os gradientes sejam acumulados entre amostras do dataset e que os resultados sejam corretos mesmo em caso de ramificações e junções no grafo computacional.

- Lembre-se de zerar os gradientes após cada passo de gradient descent (atualização dos parâmetros).


## Testes Básicos

Estes testes avaliam se a derivada da função está sendo calculada corretamente, mas em muitos casos **não** avaliam se os gradientes backpropagados estão sendo incorporados corretamente. Esta avaliação será feita nos problemas da próxima seção.

Operador de Soma

In [21]:
# add

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = add(a, b)
d = add(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=, shape=(3, 1))
Tensor([[1.]
 [1.]
 [1.]], name=, shape=(3, 1))


Operador de Subtração

In [22]:
# sub

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = sub(a, b)
d = sub(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1 e -1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=, shape=(3, 1))
Tensor([[-1.]
 [-1.]
 [-1.]], name=, shape=(3, 1))


Operador de Produto

In [23]:
# prod

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = prod(a, b)
d = prod(c, 3.0)
d.backward()

# esperado: [12, 15, 18]^T
print(a.grad)
# esperado: [3, 6, 9]^T
print(b.grad)


Tensor([[12.]
 [15.]
 [18.]], name=, shape=(3, 1))
Tensor([[3.]
 [6.]
 [9.]], name=, shape=(3, 1))


Operadores trigonométricos

In [24]:
# sin e cos

a = Tensor([np.pi, 0, np.pi/2])
b = sin(a)
c = cos(a)
d = my_sum(add(b, c))
d.backward()

# esperado: [-1, 1, -1]^T
print(a.grad)

Tensor([[-1.]
 [ 1.]
 [-1.]], name=, shape=(3, 1))


In [25]:
# Sum

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = add(prod(a, 3.0), a)
c = my_sum(b)
c.backward()

# esperado: [4, 4, 4, 4]^T
print(a.grad)


Tensor([[4.]
 [4.]
 [4.]
 [4.]], name=, shape=(4, 1))


In [26]:
# Mean

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = mean(a)
b.backward()

# esperado: [0.25, 0.25, 0.25, 0.25]^T
print(a.grad)


Tensor([[0.25]
 [0.25]
 [0.25]
 [0.25]], name=, shape=(4, 1))


In [27]:
# Square

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = square(a)

# esperado: [9, 1, 0, 4]^T
print(b)

b.backward()

# esperado: [6, 2, 0, 4]
print(a.grad)

Tensor([[9.]
 [1.]
 [0.]
 [4.]], name=square, shape=(4, 1))
Tensor([[6.]
 [2.]
 [0.]
 [4.]], name=, shape=(4, 1))


In [28]:
# matmul

W = Tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
])

v = Tensor([1.0, 2.0, 3.0])

z = matmul(W, v)

# esperado: [14, 32, 50]^T
print(z)

z.backward()

# esperado:
# [1, 2, 3]
# [1, 2, 3]
# [1, 2, 3]
print(W.grad)

# esperado: [12, 15, 18]^T
print(v.grad)


Tensor([[14.]
 [32.]
 [50.]], name=matmul, shape=(3, 1))
Tensor([[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]], name=, shape=(3, 3))
Tensor([[12.]
 [15.]
 [18.]], name=, shape=(3, 1))


In [29]:
# Exp

v = Tensor([1.0, 2.0, 3.0])
w = exp(v)

# esperado: [2.718..., 7.389..., 20.085...]^T
print(w)

w.backward()

# esperado: [2.718..., 7.389..., 20.085...]^T
print(v.grad)

Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=exp, shape=(3, 1))
Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=, shape=(3, 1))


In [30]:
# Relu

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = relu(v)

# esperado: [0, 0, 1, 3]^T
print(w)

w.backward()

# esperado: [0, 0, 1, 1]^T
print(v.grad)

Tensor([[0.]
 [0.]
 [1.]
 [3.]], name=relu, shape=(4, 1))
Tensor([[0.]
 [0.]
 [1.]
 [1.]], name=, shape=(4, 1))


In [31]:
# Sigmoid

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = sigmoid(v)

# esperado: [0.268.., 0.5, 0.731.., 0.952..]^T
print(w)

w.backward()

# esperado: [0.196..., 0.25, 0.196..., 0.045...]^T
print(v.grad)

Tensor([[0.26894142]
 [0.5       ]
 [0.73105858]
 [0.95257413]], name=sigmoid, shape=(4, 1))
Tensor([[0.19661193]
 [0.25      ]
 [0.19661193]
 [0.04517666]], name=, shape=(4, 1))


In [32]:
# Tanh

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = tanh(v)

# esperado: [[-0.76159416, 0., 0.76159416, 0.99505475]^T
print(w)

w.backward()

# esperado: [0.41997434, 1., 0.41997434, 0.00986604]^T
print(v.grad)

Tensor([[-0.76159416]
 [ 0.        ]
 [ 0.76159416]
 [ 0.99505475]], name=tanh, shape=(4, 1))
Tensor([[0.41997434]
 [1.        ]
 [0.41997434]
 [0.00986604]], name=, shape=(4, 1))


In [33]:
# Softmax

x = Tensor([-3.1, 0.5, 1.0, 2.0])
y = softmax(x)

# esperado: [0.00381737, 0.13970902, 0.23034123, 0.62613238]^T
print(y)
#
# # como exemplo, calcula o MSE para um target vector
# diff = sub(y, [1, 0, 0, 0])
# sq = square(diff)
# a = mean(sq)
#
# # esperado: 0.36424932
# print("MSE:", a)
#
# a.backward()
#
# # esperado: [-0.00278095, -0.02243068, -0.02654377, 0.05175539]^T
# print(x.grad)



Tensor([[0.00381737]
 [0.13970902]
 [0.23034123]
 [0.62613238]], name=softmax, shape=(4, 1))


## Pontos Extras

### Tarefas

- **+2 pontos**: Utilizar sobrecarga de operadores para permitir que todas as operações disponíveis aos arrays do numpy possam ser realizadas com tensores, incluindo operações que envolvam broadcasting.
  - Por exemplo, assumindo que a e b são tensores possivelmente com dimensões diferentes, devem ser possível realizar as operações a + 2, a * b, a @ b, a.max(), a.sum(axis=0).
  - Para realizar esta atividade, os atributos da classe Tensor podem ser completamente modificados, mas deve ser provido um método backward para iniciar o backpropagation.
  - Naturalmente, a regra de que tensores devem ser matrizes deve ser desconsiderada neste caso.

- **+1 ponto**: Atualizar as classes para permitir derivadas de mais alta ordem (derivadas segundas, etc.).

- **+1 ponto**: Entregar uma versão adicional do trabalho completo usando C/C++ e com foco em minimizar o tempo para realização das operações. Os casos de teste do sistema Testr também deverão ser replicados utilizando esta linguagem.

### Regras

- Só serão elegíveis para receber pontos extras os alunos que cumprirem 100% dos requisitos da parte principal do trabalho.

- Para receber os pontos extras, deverá ser agendado um horário para uma entrevista individual que abordará tanto os códigos-fonte relativos aos pontos extras quanto à parte principal do trabalho (pode acontecer redução da pontuação da parte principal do trabalho).

- Receberá os pontos extras quem responder corretamente às perguntas da entrevista. Não será atribuída pontuação parcial aos pontos extras.

## Referências

### Principais

- [Build your own pytorch](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-1-Computation-Graphs/)
- [Build your own Pytorch - 2: Backpropagation](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-2-Autograd/)
- [Build your own PyTorch - 3: Training a Neural Network with self-made AD software](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-3-Build-Classifier/)
- [Pytorch: A Gentle Introduction to torch.autograd](https://docs.pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)
- [Automatic Differentiation with torch.autograd](https://docs.pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html)

### Secundárias

- [Tom Roth: Building a computational graph: part 1](https://tomroth.dev/compgraph1/)
- [Tom Roth: Building a computational graph: part 2](https://tomroth.dev/compgraph2/)
- [Tom Roth: Building a computational graph: part 3](https://tomroth.dev/compgraph3/)
- [Roger Grosse (Toronto) class on Automatic Differentiation](https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/slides/lec10.pdf)
- [Computational graphs and gradient flows](https://simple-english-machine-learning.readthedocs.io/en/latest/neural-networks/computational-graphs.html)
- [Colah Visual Blog: Backprop](https://colah.github.io/posts/2015-08-Backprop/)
- [Towards Data Science: Automatic Differentiation (AutoDiff): A Brief Intro with Examples](https://towardsdatascience.com/automatic-differentiation-autodiff-a-brief-intro-with-examples-3f3d257ffe3b/)
- [A Hands-on Introduction to Automatic Differentiation - Part 1](https://mostafa-samir.github.io/auto-diff-pt1/)
- [Build Your own Deep Learning Framework - A Hands-on Introduction to Automatic Differentiation - Part 2](https://mostafa-samir.github.io/auto-diff-pt1/)
